In [13]:
import ngmix
import matplotlib.pyplot as plt
import galsim
import numpy as np
import numpy as np
import scipy.optimize

def biasFunc(x, m, c):
    '''Systematic bias of shear estimation
    Parameters:
    ----
    x:  input shear
    m:  multiplicative bias
    c:  additive bias

    Returns:
    ----
    y:  estimated shear
    '''
    y   =   (1+m) * x + c
    return y

def select(data, shear_type):
    """
    select the data by shear type and size
    Parameters
    ----------
    data: array
        The array with fields shear_type and T
    shear_type: str
        e.g. 'noshear', '1p', etc.
    Returns
    -------
    array of indices
    """

    w, = np.where(
        (data['flags'] == 0) & (data['shear_type'] == shear_type)
    )
    return w

def make_struct(res, obs, shear_type):
    """
    make the data structure
    Parameters
    ----------
    res: dict
        With keys 's2n', 'e', and 'T'
    obs: ngmix.Observation
        The observation for this shear type
    shear_type: str
        The shear type
    Returns
    -------
    1-element array with fields
    """
    dt = [
        ('flags', 'i4'),
        ('shear_type', 'U7'),
        ('s2n', 'f8'),
        ('g', 'f8', 2),
        ('T', 'f8'),
        ('Tpsf', 'f8'),
    ]
    data = np.zeros(1, dtype=dt)
    data['shear_type'] = shear_type
    data['flags'] = res['flags']
    if res['flags'] == 0:
        #data['s2n'] = res['s2n']
        data['s2n'] = res['s2n']
        # for moments we are actually measureing e, the elliptity
        data['g'] = res['e']
        data['T'] = res['T']
    else:
        data['s2n'] = np.nan
        data['g'] = np.nan
        data['T'] = np.nan
        data['Tpsf'] = np.nan

        # we only have one epoch and band, so we can get the psf T from the
        # observation rather than averaging over epochs/bands
        data['Tpsf'] = obs.psf.meta['result']['T']
    return data

In [16]:
def make_data(rng, FbyB, shear, version=0, first=False, SbyN=20):
    """
    simulate an exponential object with moffat psf
    Parameters
    ----------
    rng: np.random.RandomState
        The random number generator
    noise: float
        Noise for the image
    FbyB: float
        source by background ratio
    shear: (g1, g2)
        The shear in each component
    Returns
    -------
    ngmix.Observation
    """

    scale    = 0.263
    psf_fwhm = 0.9
    gal_hlr  = 0.5
    dy, dx   = rng.uniform(low=-scale/2, high=scale/2, size=2)

    psf = galsim.Moffat(beta=2.5, fwhm=psf_fwhm,
    ).shear(g1=0.02, g2=-0.01,)
    
    
    obj0 = galsim.Exponential(
        half_light_radius=gal_hlr, flux=125e3
    ).shear(
        g1=shear,
        g2=0,
    ).shift(dx=dx, dy=dy,)
    obj = galsim.Convolve(psf, obj0)

    psf_im = psf.drawImage(scale=scale).array
    im = obj.drawImage(scale = scale)
 
    # psf noise
    psf_noise= 1e-9
    psf_im += rng.normal(scale=psf_noise, size=psf_im.shape)
    
    ngrid = im.array.shape[0]
    flux_tmp = np.sum((im.array)[ngrid//2-2:ngrid//2+3, ngrid//2-2:ngrid//2+3])
    std_tmp =  5
    F = SbyN**2.*(1+FbyB)/FbyB
    B = F/FbyB
    B_val = B/25.0
    F_val=F/25.0
        
    im = (im/flux_tmp)*F
    
    
    if version==0:
        noise_image = rng.normal(scale=1, size=im.array.shape)
        noise_image *= np.sqrt(B_val + im.array)/std_tmp
        variance_array = np.ones_like(im.array)*(B_val+im.array)/std_tmp**2
        im += noise_image
            
    if version==1:
        noise_image = rng.normal(scale=1, size=im.array.shape)
        noise_image *= np.sqrt(B_val+F_val)/std_tmp
        variance_array = np.ones_like(im.array)*(B_val+F_val)/std_tmp**2
        im += noise_image

        
    imArr = im.array
    
    cen = (np.array(imArr.shape)-1.0)/2.0
    psf_cen = (np.array(psf_im.shape)-1.0)/2.0
    jacobian = ngmix.DiagonalJacobian(
        row=cen[0] + dy/scale, col=cen[1] + dx/scale, scale=scale,
    )
    psf_jacobian = ngmix.DiagonalJacobian(
        row=psf_cen[0], col=psf_cen[1], scale=scale,
    )

    # noise variance map
    if version==0:
        wt = 1.0/np.average(variance_array)*np.ones_like(variance_array)
    if version==1:
        wt = 1.0/variance_array
        
    psf_wt = psf_im*0 + 1.0/psf_noise**2

    psf_obs = ngmix.Observation(
        psf_im,
        weight=psf_wt,
        jacobian=psf_jacobian,
    )
    obs = ngmix.Observation(
        imArr,
        weight=wt,
        jacobian=jacobian,
        psf=psf_obs,
    )
    return obs

In [17]:
def analyze(rng, num_tests, num_gals, FBratioArr, shear_true, version=0, first=False, SbyN=20):
    data = []
    x = []
    y = []
    shear_error = []
    
    for i in range(num_tests):
        print("len(data[]) =", len(data))
        dlist = []
        
        for j in range(num_gals):
            imgdata = make_data(rng=rng, FbyB=FBratioArr[i], shear=shear_true[i], version=version, first=first, SbyN=20)
            obs = imgdata

            resdict, obsdict = boot.go(obs)
            for stype, sres in resdict.items():
                st = make_struct(res=sres, obs=obsdict[stype], shear_type=stype)
                dlist.append(st)

        data.append(np.hstack(dlist))
        #data contains average of all galaxy data

        w = select(data=data[i], shear_type='noshear')
        w_1p = select(data=data[i], shear_type='1p')
        w_1m = select(data=data[i], shear_type='1m')
        g_1p = data[i]['g'][w_1p, 0].mean()
        g_1m = data[i]['g'][w_1m, 0].mean()
        R11 = (g_1p - g_1m)/0.02

        g = data[i]['g'][w].mean(axis=0)
        shear = g / R11

        g_error = data[i]['g'][w].std(axis=0) / np.sqrt(w.size)
        shear_error.append(g_error[0]/R11)

        x.append(FBratioArr[i])
        y.append(shear[0])

            
    return (x, y, shear_error)

In [18]:
rng = np.random.RandomState(1024)
#rng = galsim.BaseDeviate(1024)

# We will measure moments with a fixed gaussian weight function
weight_fwhm= 1.2
fitter     = ngmix.gaussmom.GaussMom(fwhm=weight_fwhm)
psf_fitter = ngmix.gaussmom.GaussMom(fwhm=weight_fwhm)

# these "runners" run the measurement code on observations
psf_runner = ngmix.runners.PSFRunner(fitter=psf_fitter)
runner     = ngmix.runners.Runner(fitter=fitter)

# this "bootstrapper" runs the metacal image shearing as well as both psf
# and object measurements
#
# We will just do R11 for simplicity and to speed up this example;
# typically the off diagonal terms are negligible, and R11 and R22 are
# usually consistent
boot      = ngmix.metacal.MetacalBootstrapper(
    runner= runner, psf_runner=psf_runner,
    rng=rng,
    psf='gauss',
    types=['noshear', '1p', '1m'],
)

In [43]:
FBratioArr = np.logspace(start=-2, stop=2, num=5, base=10.0)
num_tests = len(FBratioArr)
num_gals = 100
g_true= [-0.03, -0.01, 0, 0.01, 0.03]

analysis_BandS1 = analyze(rng, num_tests, num_gals, FBratioArr, g_true, version=0, first = True, SbyN=20)
analysis_BandS2 = analyze(rng, num_tests, num_gals, FBratioArr, g_true, version=0, first = True, SbyN=20)

analysis_Bonly1 = analyze(rng, num_tests, num_gals, FBratioArr, g_true, version=1, first = True, SbyN=20)
analysis_Bonly2 = analyze(rng, num_tests, num_gals, FBratioArr, g_true, version=1, first = True, SbyN=20)

len(data[]) = 0
len(data[]) = 1
len(data[]) = 2
len(data[]) = 3
len(data[]) = 4
len(data[]) = 0
len(data[]) = 1
len(data[]) = 2
len(data[]) = 3
len(data[]) = 4
len(data[]) = 0
len(data[]) = 1
len(data[]) = 2
len(data[]) = 3
len(data[]) = 4
len(data[]) = 0
len(data[]) = 1
len(data[]) = 2
len(data[]) = 3
len(data[]) = 4


In [57]:
#popt, pcov = scipy.optimize.curve_fit(biasFunc, shear_true[0], shear[0], sigma = shear_error[0])

x = np.linspace(-.03*1.5, .03*1.5, len(FBratioArr))

FbyBlist = analysis_BandS1[0]

"""
print("analysis_BandS1", analysis_BandS1[1])
print("analysis_BandS2", analysis_BandS2[1])
print("analysis_Bonly1", analysis_Bonly1[1])
print("analysis_Bonly2", analysis_Bonly2[1])
"""

g11_est = analysis_BandS1[1]
g11_sigma = analysis_BandS1[2]

g12_est = analysis_BandS2[1]
g12_sigma = analysis_BandS2[2]

g21_est = analysis_Bonly1[1]
g21_sigma = analysis_Bonly1[2]

g22_est = analysis_Bonly2[1]
g22_sigma = analysis_Bonly2[2]

"""
print("g11_est =", g11_est)
print("g12_est =", g12_est)

print("g21_est =", g21_est)
print("g22_est =", g22_est)

print("g_true =", g_true)

print("g11_sigma =", g11_sigma)
print("g12_sigma =", g12_sigma)

print("g21_sigma =", g21_sigma)
print("g22_sigma =", g22_sigma)
"""

for n in range(len(g_true)):
    tempBiasV0 = []
    tempBiasV1 = []
    tempBiasErrorsV0 = []
    tempBiasErrorsV1 = []
    biasList = []
    errorList = []
    
    tempBiasV0.append(g11_est[n])
    tempBiasV0.append(g12_est[n])
    
    tempBiasV1.append(g21_est[n])
    tempBiasV1.append(g22_est[n])
    
    tempBiasErrorsV0.append(g11_sigma[n])
    tempBiasErrorsV0.append(g12_sigma[n])
    
    tempBiasErrorsV1.append(g21_sigma[n])
    tempBiasErrorsV1.append(g22_sigma[n])
    
    
    """
    print("shape(tempBiasV0)", np.shape(tempBiasV0))
    print("shape(tempBiasV1)", np.shape(tempBiasV1))
    print("shape(tempBiasErrorsV0)", np.shape(tempBiasErrorsV0))
    print("shape(tempBiasErrorsV1)", np.shape(tempBiasErrorsV1))
    """

    print("g_true[n] =", g_true[n])
    poptV0, pcovV0 = scipy.optimize.curve_fit(biasFunc, g_true[n], tempBiasV0, sigma = tempBiasErrorsV0)
    print("tempBiasV0 = ",tempBiasV0)
    print("tempBiasErrorsV0 = ",tempBiasErrorsV0)
    print("poptV0 = ",poptV0)
    poptV1, pcovV1 = scipy.optimize.curve_fit(biasFunc, g_true[n], tempBiasV1, sigma = tempBiasErrorsV1)
    print("tempBiasV1 = ",tempBiasV1)
    print("tempBiasErrorsV1 = ",tempBiasErrorsV1)
    print("poptV1 = ",poptV1)
    print("\n")
    
    biasList.append(poptV0)
    biasList.append(poptV1)
    
    errorList.append(pcovV0)
    errorList.append(pcovV1)

print("biasList =", biasList)
print("errorList =", errorList)

"""
plt.figure(figsize=(20, 10))
plt.errorbar(np.array(g_true)*1.1, g1_est, yerr=g2_sigma, fmt='o', label='Background + Source')
plt.errorbar(g_true, g2_est, yerr=g2_sigma, fmt='o', label='Background only')
#plt.plot(x, y, color="purple")
plt.plot(np.array(x), biasFunc(np.array(x), poptV0[0], poptV0[1]), label = 'Background + Source', color = "blue")
plt.plot(np.array(x), biasFunc(np.array(x), poptV1[0], poptV1[1]), label = 'Background only', color = "orange")
plt.axhline(0, color="purple")
plt.axvline(0, color="purple")
plt.xlabel(r'$g_{true}$', fontsize=16)
plt.ylabel(r'$g_{est}$', fontsize=16)
plt.legend()
"""

g_true[n] = -0.03
tempBiasV0 =  [-0.03226904112005742, -0.025103348666702593]
tempBiasErrorsV0 =  [0.0031974252826337214, 0.0028273459193107887]
poptV0 =  [-6008.34297234  -180.24853695]
tempBiasV1 =  [-0.030157989063746036, -0.031959299645932776]
tempBiasErrorsV1 =  [0.0032111764575349804, 0.0029966472834220825]
poptV1 =  [6548.33657457  196.44897647]


g_true[n] = -0.01
tempBiasV0 =  [-0.010141526050268143, -0.008574848353411877]
tempBiasErrorsV0 =  [0.0029238947165782228, 0.0031089624788839705]
poptV0 =  [51533.08579691   515.33145179]
tempBiasV1 =  [-0.008140369636702722, -0.007986874656557568]
tempBiasErrorsV1 =  [0.0031287385701920223, 0.003097712155051114]
poptV1 =  [88226.44030164   882.26634016]


g_true[n] = 0
tempBiasV0 =  [0.0015218353345612947, 0.0012630488142850854]
tempBiasErrorsV0 =  [0.0023191330122909128, 0.002686198538804608]
poptV0 =  [1.         0.00141132]
tempBiasV1 =  [-0.000973257719325112, 0.004792399061257314]
tempBiasErrorsV1 =  [0.003027086938637535, 0.0027

'\nplt.figure(figsize=(20, 10))\nplt.errorbar(np.array(g_true)*1.1, g1_est, yerr=g2_sigma, fmt=\'o\', label=\'Background + Source\')\nplt.errorbar(g_true, g2_est, yerr=g2_sigma, fmt=\'o\', label=\'Background only\')\n#plt.plot(x, y, color="purple")\nplt.plot(np.array(x), biasFunc(np.array(x), poptV0[0], poptV0[1]), label = \'Background + Source\', color = "blue")\nplt.plot(np.array(x), biasFunc(np.array(x), poptV1[0], poptV1[1]), label = \'Background only\', color = "orange")\nplt.axhline(0, color="purple")\nplt.axvline(0, color="purple")\nplt.xlabel(r\'$g_{true}$\', fontsize=16)\nplt.ylabel(r\'$g_{est}$\', fontsize=16)\nplt.legend()\n'

In [ ]:
fig, (p1, p2) = plt.subplots(2, figsize=(5,25))

print(np.shape(FbyBlist))
print(biasVals0)

plot1 = p1.errorbar(np.array(FbyBlist)*1.1, biasVals0[0], np.sqrt(np.diag(biasError0)), fmt='o', label='Background + Source')
p1.errorbar(FbyBlist, biasVals1[0], np.average(biasError1[0]), fmt='o', label='Background only')
p1.set_title("Multiplicative Bias Plot")
p1.axhline(0, color="purple")
p1.xlabel(r'$\frac{F}{B}$', fontsize=16)
p1.ylabel(r'$m$', fontsize=16)
p1.xscale("log")


plot2 = p2.errorbar(np.array(FbyBlist)*1.1, biasVals0[1], np.average(biasError0[1]), fmt='o', label='Background + Source')
p2.errorbar(FbyBlist, biasVals1[1], np.average(biasError1[1]), fmt = 'o', label ='Background only')
p2.axhline(0, color="purple")
p2.xlabel(r'$\frac{F}{B}$', fontsize=16)
p2.ylabel(r'$c$', fontsize=16)
p2.xscale("log")
p2.set_title("Additive Bias Plot")

plt.legend()
plt.show()